# Create ISA-API Investigation from Datascriptor Study Design configuration

In this notebook, we will show how to use an **ISA study design configuration** in *JSON format*, as generated by **[DataScriptor](https://gitlab.com/datascriptor/datascriptor)**  to create a fully fledged single-study ISA investigation and then how to serialise it either in JSON or tabular (i.e. CSV) format.

Our study design configuration consists of:

- a `longitudinal intervention study` with 3 compound based `treatments` over 4 (out of 6 possible) `arms` in a `crossover design`.

    - Treatment A: aspirin, 5 mg, 10 days
    - Treatment B: ibuprofen, 8 mg, 10 days
    - Treatment C: paracetamol, 10 mg, 10 days   
    - Possible Treatment sequences: ["ABC","ACB","BAC","BCA","CAB","CBA"]
- each `arm` receives *N=5* `study subjects`
- each `study subject` is used to collect `samples` (blood and saliva). 
    - blood was collected once (nB=1) at the end of each sequence in the 4 arms used.
    - saliva was collected four times (nS=4).
- `samples` are analysed with several distinct `data acquisition modalities`:
    
    - `metabolite profiling using mass spectrometry`   

- `technical replicates` *(n=2)* are obtained for mass spectrometry based acquisitions.
- no technical replication was performed for nmr spectroscopy based acquisitions. 


## 1. Getting ready

Let's start by importing all the necessary libraries to perform the tasks.

In [17]:
from time import time
import os
import json

## ISA-API related imports
from isatools.model import Investigation, Study

## ISA-API create mode related imports
from isatools.create import models
from isatools.create.models import StudyDesign
from isatools.create.connectors import generate_study_design_from_config

# serializer from ISA Investigation to JSON
from isatools.isajson import ISAJSONEncoder

# ISA-Tab serialisation
from isatools import isatab

## ISA-JSON serialisation
from isatools import isajson

## 2. Reading the `ISA Study Design JSON configuration` created by DataScriptor

The first step is to read in the study design information saved as JSON by DataScritor

In [18]:
with open(os.path.abspath(os.path.join("ds-study-design-config", "study-design-3-repeated-treatments-datascriptor.json")), "r") as config_file:
    study_design_config = json.load(config_file)

Let's now view the content of this variable:

In [19]:
study_design_config

{'treatmentPlan': {'screen': {'name': 'screen',
   'duration': None,
   'durationUnit': ''},
  'runIn': {'name': 'run-in', 'duration': None, 'durationUnit': ''},
  'washout': {'name': 'washout', 'duration': '5', 'durationUnit': 'days'},
  'followUp': {'name': 'follow-up', 'duration': 60, 'durationUnit': 'days'},
  'treatments': [{'agent': 'ibuprofen',
    'intensity': 8,
    'intensityUnit': 'mg',
    'duration': 10,
    'durationUnit': 'days'},
   {'agent': 'aspirin',
    'intensity': 5,
    'intensityUnit': 'mg',
    'duration': 10,
    'durationUnit': 'days'},
   {'agent': 'paracetamol',
    'intensity': 10,
    'intensityUnit': 'mg',
    'duration': 10,
    'durationUnit': 'days'}],
  'elementParams': {'agents': [],
   'intensities': [],
   'intensityUnit': '',
   'durations': [],
   'durationUnit': ''}},
 'generatedStudyDesign': {'name': 'test study',
  'description': 'this is the verbose description',
  'type': {'term': 'crossover design',
   'id': 'OBI:0500003',
   'iri': 'http:

## 3. Generating the `ISA Study Design` objects  from the DataScriptor JSON configuration
To perform this conversion,  we simply need to invoke the function `generate_isa_study_design_from_config` (name possibly subject to change, should we drop the "isa" and "datascriptor" qualifiers?)

In [20]:
study_design = generate_study_design_from_config(study_design_config)
assert isinstance(study_design, StudyDesign)

In [21]:
study_design

isatools.create.models.StudyDesign(name=Study Design, study_arms=[isatools.create.models.StudyArm(name=Arm_0, source_type=human, group_size=5, cells=[isatools.create.models.StudyCell(name=CELL_Arm_0_0, elements=[isatools.create.models.Treatment(type=unspecified intervention, factor_values=[isatools.model.FactorValue(factor_name=isatools.model.StudyFactor(name='AGENT', factor_type=isatools.model.OntologyAnnotation(term='perturbation agent', term_source=None, term_accession='', comments=[]), comments=[]), value='ibuprofen', unit=None), isatools.model.FactorValue(factor_name=isatools.model.StudyFactor(name='DURATION', factor_type=isatools.model.OntologyAnnotation(term='time', term_source=None, term_accession='', comments=[]), comments=[]), value=10, unit='days'), isatools.model.FactorValue(factor_name=isatools.model.StudyFactor(name='INTENSITY', factor_type=isatools.model.OntologyAnnotation(term='intensity', term_source=None, term_accession='', comments=[]), comments=[]), value=8, unit='m

## 4. Generate the ISA Study from the StudyDesign and embed it into an ISA Investigation

The `StudyDesign.generate_isa_study()` method returns the complete ISA-API `Study` object.

In [22]:
start = time() #this is simply to estimate how long the process will take so we start the clock

study = study_design.generate_isa_study()

end = time() #we stop the clock

print('The generation of the study design took {:.2f} s.'.format(end - start))
assert isinstance(study, Study)


The generation of the study design took 0.24 s.


We now simply create an ISA investigation object by adding the newly created study.

In [23]:
investigation = Investigation(studies=[study])

## 5. Writing the generated ISA Investigation to file in ISA-JSON format

In [24]:
start = time() #this is simply to estimate how long the process will take so we start the clock

inv_json = json.dumps(investigation, cls=ISAJSONEncoder, sort_keys=True, indent=4, separators=(',', ': '))

end = time()#we stop the clock
print('The JSON serialisation of the ISA investigation took {:.2f} s.'.format(end - start))

The JSON serialisation of the ISA investigation took 0.09 s.


Writing the ISA JSON serialization to file:

In [25]:
directory = os.path.abspath(os.path.join('output'))
if not os.path.exists(directory):
    os.makedirs(directory)
with open(os.path.abspath(os.path.join('output','isa-investigation-3-arms-ms-only.json')), 'w') as out_fp:
    json.dump(json.loads(inv_json), out_fp)

## 6. Writing the same ISA Investigation to file in ISA-TAB format

In [26]:
start = time()
isatab.dump(investigation, os.path.abspath(os.path.join('output')))
end = time()
print('The Tab serialisation of the ISA investigation took {:.2f} s.'.format(end - start))

2020-10-31 12:39:22,336 [INFO]: isatab.py(_all_end_to_end_paths:1152) >> Found 100 paths!
2020-10-31 12:39:22,346 [INFO]: isatab.py(write_study_table_files:1330) >> Rendered 100 paths
2020-10-31 12:39:22,351 [INFO]: isatab.py(write_study_table_files:1337) >> Writing 100 rows
2020-10-31 12:39:22,476 [INFO]: isatab.py(_all_end_to_end_paths:1152) >> Found 20 paths!
2020-10-31 12:39:22,601 [INFO]: isatab.py(_all_end_to_end_paths:1152) >> Found 20 paths!
2020-10-31 12:39:22,728 [INFO]: isatab.py(_all_end_to_end_paths:1152) >> Found 20 paths!
2020-10-31 12:39:22,971 [INFO]: isatab.py(_all_end_to_end_paths:1152) >> Found 40 paths!
2020-10-31 12:39:23,100 [INFO]: isatab.py(_all_end_to_end_paths:1152) >> Found 20 paths!
2020-10-31 12:39:23,229 [INFO]: isatab.py(_all_end_to_end_paths:1152) >> Found 20 paths!
2020-10-31 12:39:23,362 [INFO]: isatab.py(_all_end_to_end_paths:1152) >> Found 20 paths!
2020-10-31 12:39:23,598 [INFO]: isatab.py(_all_end_to_end_paths:1152) >> Found 40 paths!
2020-10-31 1

The Tab serialisation of the ISA investigation took 3.12 s.


In [27]:
#%debug

To use them on the notebook we can also dump the tables to pandas DataFrames, using the `dump_tables_to_dataframes` function rather than dump

In [28]:
dataframes = isatab.dump_tables_to_dataframes(investigation)

2020-10-31 12:39:25,502 [INFO]: isatab.py(_all_end_to_end_paths:1152) >> Found 100 paths!
2020-10-31 12:39:25,509 [INFO]: isatab.py(write_study_table_files:1330) >> Rendered 100 paths
2020-10-31 12:39:25,514 [INFO]: isatab.py(write_study_table_files:1337) >> Writing 100 rows
2020-10-31 12:39:25,638 [INFO]: isatab.py(_all_end_to_end_paths:1152) >> Found 20 paths!
2020-10-31 12:39:25,767 [INFO]: isatab.py(_all_end_to_end_paths:1152) >> Found 20 paths!
2020-10-31 12:39:25,890 [INFO]: isatab.py(_all_end_to_end_paths:1152) >> Found 20 paths!
2020-10-31 12:39:26,147 [INFO]: isatab.py(_all_end_to_end_paths:1152) >> Found 40 paths!
2020-10-31 12:39:26,288 [INFO]: isatab.py(_all_end_to_end_paths:1152) >> Found 20 paths!
2020-10-31 12:39:26,422 [INFO]: isatab.py(_all_end_to_end_paths:1152) >> Found 20 paths!
2020-10-31 12:39:26,556 [INFO]: isatab.py(_all_end_to_end_paths:1152) >> Found 20 paths!
2020-10-31 12:39:26,806 [INFO]: isatab.py(_all_end_to_end_paths:1152) >> Found 40 paths!
2020-10-31 1

In [29]:
len(dataframes)

17

## 7. Check the correctness of the ISA-Tab DataFrames 

We have 1 study file and 16 assay files:
* one isa assay table is currently generated for each `cell` defined in the `arm/epoch` coordinate system.
* in this design example, 4 arms are used and sampling event occur in 4 distinct epochs

NB: This behavior is being changed and in the future release, all data acquisitions for a given assay type will be grouped in a single ISA assay table.


In [30]:
for key in dataframes.keys():
    display(key)

's_study_01.txt'

'a_CELL_Arm_4_4_ASSAY_GRAPH_000_metabolite-profiling_mass-spectrometry.txt'

'a_CELL_Arm_0_5_ASSAY_GRAPH_000_metabolite-profiling_mass-spectrometry.txt'

'a_CELL_Arm_3_0_ASSAY_GRAPH_000_metabolite-profiling_mass-spectrometry.txt'

'a_CELL_Arm_4_2_ASSAY_GRAPH_000_metabolite-profiling_mass-spectrometry.txt'

...

'a_CELL_Arm_3_2_ASSAY_GRAPH_000_metabolite-profiling_mass-spectrometry.txt'

'a_CELL_Arm_2_0_ASSAY_GRAPH_000_metabolite-profiling_mass-spectrometry.txt'

'a_CELL_Arm_2_5_ASSAY_GRAPH_000_metabolite-profiling_mass-spectrometry.txt'

'a_CELL_Arm_2_4_ASSAY_GRAPH_000_metabolite-profiling_mass-spectrometry.txt'

'a_CELL_Arm_2_2_ASSAY_GRAPH_000_metabolite-profiling_mass-spectrometry.txt'

We have 5 subjects in each arm and 5 samples per subject have been collected (4 saliva samples per subject and 1 blood sample per subject)

In [33]:
study_frame = dataframes['s_study_01.txt']
# count_control_samples = len(study_frame[study_frame['Source Name'].apply(lambda el: 'control' in el)])
# count_treatment_samples = len(study_frame[study_frame['Source Name'].apply(lambda el: 'treatment' in el)])
# print("There are {} samples in the control arm (i.e. group)".format(count_control_samples))
# print("There are {} samples in the treatment arm (i.e. group)".format(count_treatment_samples))

Each control samples is fractioned and 2 labelled extracts are produced (i.e. biological replicates).

```
[
    "labelling",
    {
        "#replicates": 2
    }
]
```

There are 2 possible combinations of mass spectrometry acquisition specified in this study design configuration template and 2 technical replicates are produced for each combinations:

```
[
    "mass spectrometry",
    {
        "#replicates": 2,
        "instrument": [
            "Agilent QTQF 6510"
        ],
        "injection_mode": [
            "FIA",
            "LC"
        ],
        "acquisition_mode": [
            "positive mode"
        ]
    }
]
```

Two output raw spectral files are produced as the result of reach run

```
[
    "raw spectral data file",
    [
        {
            "node_type": "data file",
            "size": 2,
            "is_input_to_next_protocols": false
        }
    ]
]
```

As a total, we expect:

$$ N_{rows} = (N_{subjects} \times N_{sample_type}) \times (N_{biorepl} \times N_{combinations} \times N_{techrepl}) = (5 \times 2) \times (2 \times 1 \times 1 \times 2) = 10 \times 4 = 40 $$

We can verify for the mass spectrometry assay file of the last epoch of the first arm of the study.

In [32]:
dataframes['a_CELL_Arm_0_5_ASSAY_GRAPH_000_metabolite-profiling_mass-spectrometry.txt']

,Sample Name,Protocol REF,Performer,Extract Name,Characteristics[extract type],Protocol REF.1,Performer.1,Labeled Extract Name,Label,Protocol REF.2,Parameter Value[instrument],Parameter Value[injection_mode],Parameter Value[acquisition_mode],MS Assay Name,Performer.2,Raw Spectral Data File
0,GRP-Arm_0.SBJ-001.CEL-CELL_Arm_0_5.SMP-.blood.1,extraction,Ellipsis,extract_2-1,polar fraction,labeling,Ellipsis,labeled extract_2-3,biotin,mass spectrometry,Agilent QTQF 6510,LC,positive mode,mass spectrometry_2-4,Ellipsis,raw spectral data file_2-5
1,GRP-Arm_0.SBJ-001.CEL-CELL_Arm_0_5.SMP-.blood.1,extraction,Ellipsis,extract_2-1,polar fraction,labeling,Ellipsis,labeled extract_2-3,biotin,mass spectrometry,Agilent QTQF 6510,LC,negative mode,mass spectrometry_2-8,Ellipsis,raw spectral data file_2-9
2,GRP-Arm_0.SBJ-001.CEL-CELL_Arm_0_5.SMP-.blood.1,extraction,Ellipsis,extract_2-1,polar fraction,labeling,Ellipsis,labeled extract_2-3,biotin,mass spectrometry,Agilent QTQF 6510,LC,negative mode,mass spectrometry_2-10,Ellipsis,raw spectral data file_2-11
3,GRP-Arm_0.SBJ-001.CEL-CELL_Arm_0_5.SMP-.blood.1,extraction,Ellipsis,extract_2-1,polar fraction,labeling,Ellipsis,labeled extract_2-3,biotin,mass spectrometry,Agilent QTQF 6510,LC,positive mode,mass spectrometry_2-6,Ellipsis,raw spectral data file_2-7
4,GRP-Arm_0.SBJ-001.CEL-CELL_Arm_0_5.SMP-.saliva.1,extraction,Ellipsis,extract_7-1,polar fraction,labeling,Ellipsis,labeled extract_7-3,biotin,mass spectrometry,Agilent QTQF 6510,LC,negative mode,mass spectrometry_7-10,Ellipsis,raw spectral data file_7-11
5,GRP-Arm_0.SBJ-001.CEL-CELL_Arm_0_5.SMP-.saliva.1,extraction,Ellipsis,extract_7-1,polar fraction,labeling,Ellipsis,labeled extract_7-3,biotin,mass spectrometry,Agilent QTQF 6510,LC,positive mode,mass spectrometry_7-4,Ellipsis,raw spectral data file_7-5
6,GRP-Arm_0.SBJ-001.CEL-CELL_Arm_0_5.SMP-.saliva.1,extraction,Ellipsis,extract_7-1,polar fraction,labeling,Ellipsis,labeled extract_7-3,biotin,mass spectrometry,Agilent QTQF 6510,LC,positive mode,mass spectrometry_7-6,Ellipsis,raw spectral data file_7-7
7,GRP-Arm_0.SBJ-001.CEL-CELL_Arm_0_5.SMP-.saliva.1,extraction,Ellipsis,extract_7-1,polar fraction,labeling,Ellipsis,labeled extract_7-3,biotin,mass spectrometry,Agilent QTQF 6510,LC,negative mode,mass spectrometry_7-8,Ellipsis,raw spectral data file_7-9
8,GRP-Arm_0.SBJ-002.CEL-CELL_Arm_0_5.SMP-.blood.1,extraction,Ellipsis,extract_3-1,polar fraction,labeling,Ellipsis,labeled extract_3-3,biotin,mass spectrometry,Agilent QTQF 6510,LC,positive mode,mass spectrometry_3-4,Ellipsis,raw spectral data file_3-5
9,GRP-Arm_0.SBJ-002.CEL-CELL_Arm_0_5.SMP-.blood.1,extraction,Ellipsis,extract_3-1,polar fraction,labeling,Ellipsis,labeled extract_3-3,biotin,mass spectrometry,Agilent QTQF 6510,LC,negative mode,mass spectrometry_3-8,Ellipsis,raw spectral data file_3-9
